In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta

In [2]:
events = pd.read_csv("events.csv.gzip", compression='gzip')

/home/cecilia/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
events.isnull().sum()

index                       0
date                        0
event_id                    0
ref_type                    0
ref_hash                    0
application_id              0
attributed                  0
device_countrycode          0
device_os_version     5411606
device_brand          5191157
device_model          2076489
device_city           5849646
session_user_agent      42280
trans_id              7706939
user_agent            4403098
event_uuid              29772
carrier               5818680
kind                    29772
device_os             5874391
wifi                        0
connection_type       5935285
ip_address                  0
device_language       2079172
dtype: int64

In [4]:
# Elimino las columnas que tienen mas de la mitad de los datos nulos
events = events.drop(['device_os_version','device_brand','device_city',
                           'trans_id','user_agent','carrier','device_os','connection_type'], axis=1)

In [14]:
events['event_id'].value_counts().head(10)

1      1281105
15     1005533
23      702708
2       621588
115     524729
0       404736
13      260978
7       256957
5       250572
101     218849
Name: event_id, dtype: int64

In [7]:
events['ref_type'].value_counts()

1891515180541284343    6421584
1494519392962156891    1322997
Name: ref_type, dtype: int64

In [15]:
events['application_id'].value_counts().head(10)

210    1900636
122    1021538
121     657386
65      654725
26      491855
269     276453
155     262531
21      257102
133     247082
226     222076
Name: application_id, dtype: int64

In [9]:
events['attributed'].value_counts()

False    7714809
True       29772
Name: attributed, dtype: int64

In [10]:
events['device_countrycode'].value_counts()

6287817205707153877    7744581
Name: device_countrycode, dtype: int64

In [16]:
events['kind'].value_counts().head(10)

4.017674e+18    1281105
5.500848e+18    1005533
6.081165e+18     683828
6.168309e+18     621588
6.123834e+18     524598
5.882882e+18     404733
4.853173e+18     260978
9.977660e+17     256493
9.208056e+18     250572
6.897685e+18     214713
Name: kind, dtype: int64

In [17]:
events['wifi'].value_counts()

False    5478103
True     2266478
Name: wifi, dtype: int64

In [23]:
events['device_language'].value_counts().head(10)

3.301378e+18    2588856
6.977049e+18    2311474
8.441417e+18     486372
6.584214e+18     101223
4.060930e+18      62155
5.221863e+18      52326
7.528974e+18      20169
3.491909e+18      14824
3.177265e+18       4668
4.077062e+17       1917
Name: device_language, dtype: int64

In [27]:
def crear_features_events(events_entrenar,datetime_inicio):
    events_entrenar['date'] = pd.to_datetime(events_entrenar['date'])
    
    dia_en_segundos = 86400
    events_entrenar['segundos'] = ((events_entrenar['date']-datetime_inicio).dt.seconds + 
                                     (events_entrenar['date']-datetime_inicio).dt.days*dia_en_segundos)

    
    # Cambio sevent_id por la probabilidad de que aparezca ese evento
    evento = events_entrenar['event_id'].value_counts(normalize=True)
    events_entrenar['event_id'] = events_entrenar['event_id'].replace(evento)
    
    # Cambio application_id por la probabilidad de que aparezca esa aplicacion
    aplicacion = events_entrenar['application_id'].value_counts(normalize=True)
    events_entrenar['application_id'] = events_entrenar['application_id'].replace(aplicacion)
      
    # Cambio device_language por la probabilidad de que aparezca ese kind
    kind = events_entrenar['kind'].value_counts(normalize=True)
    events_entrenar['kind'] = events_entrenar['kind'].replace(kind)
    

    # Agrupo el data frame por los ref_hash
    events_entrenar = events_entrenar.groupby('ref_hash').agg({'ref_hash':'count','attributed':'sum',
                                                               'wifi':'sum','event_id':'mean',
                                                               'application_id':'mean','kind':'mean',
                                                               'segundos':'mean'})
    
    events_entrenar.columns = ['e_kind','e_app_id','e_event_id','e_#wifi','e_#atribuidos','e_#eventos','e_t_prom']
    events_entrenar = events_entrenar.reset_index('ref_hash')
          
    return events_entrenar
    

In [28]:
events_entrenar = pd.read_csv('events_1820.csv')
datetime_inicio = datetime(2019,4,18)
f_events = crear_features_events(events_entrenar,datetime_inicio)
f_events.to_csv('events_train_1820.csv', index=False)
f_events.head()

,ref_hash,e_kind,e_app_id,e_event_id,e_#wifi,e_#atribuidos,e_#eventos,e_t_prom
0,41863526108385,0.126640,0.211054,0.126202,0.0,0.0,88,140062.647727
1,161514654074162,0.057634,0.087007,0.058797,8.0,0.0,8,147937.250000
2,186034136943920,0.148482,0.253877,0.147929,0.0,0.0,13,60125.769231
3,360710529886978,0.064439,0.016072,0.065793,0.0,0.0,3,214874.000000
4,365882020742330,0.099017,0.019018,0.098669,0.0,0.0,57,151254.982456


In [29]:
events_entrenar = pd.read_csv('events_1921.csv')
datetime_inicio = datetime(2019,4,19)
f_events = crear_features_events(events_entrenar,datetime_inicio)
f_events.to_csv('events_train_1921.csv', index=False)
f_events.head()

,ref_hash,e_kind,e_app_id,e_event_id,e_#wifi,e_#atribuidos,e_#eventos,e_t_prom
0,40621409780134,0.005631,0.005478,0.005612,0.0,0.0,6,242807.833333
1,41863526108385,0.147983,0.261183,0.147484,0.0,0.0,72,88923.736111
2,161514654074162,0.056457,0.083004,0.057539,8.0,0.0,8,61537.250000
3,360710529886978,0.070222,0.015924,0.071648,0.0,0.0,4,155603.000000
4,365882020742330,0.099317,0.019578,0.098997,0.0,0.0,65,86141.076923


In [30]:
events_entrenar = pd.read_csv('events_2022.csv')
datetime_inicio = datetime(2019,4,20)
f_events = crear_features_events(events_entrenar,datetime_inicio)
f_events.to_csv('events_train_2022.csv', index=False)
f_events.head()

,ref_hash,e_kind,e_app_id,e_event_id,e_#wifi,e_#atribuidos,e_#eventos,e_t_prom
0,40621409780134,0.005677,0.005505,0.005659,0.0,0.0,6,156407.833333
1,41863526108385,0.146337,0.262990,0.145857,0.0,0.0,51,31455.568627
2,161514654074162,0.091154,0.088657,0.091189,50.0,0.0,51,241464.294118
3,168103949904656,0.090038,0.035244,0.091922,0.0,0.0,10,173599.100000
4,186034136943920,0.133386,0.262990,0.132950,0.0,0.0,28,237222.571429


In [31]:
events_entrenar = pd.read_csv('events_2123.csv')
datetime_inicio = datetime(2019,4,21)
f_events = crear_features_events(events_entrenar,datetime_inicio)
f_events.to_csv('events_train_2123.csv', index=False)
f_events.head()

,ref_hash,e_kind,e_app_id,e_event_id,e_#wifi,e_#atribuidos,e_#eventos,e_t_prom
0,40621409780134,0.008657,0.005934,0.008633,0.0,0.0,9,122304.666667
1,41863526108385,0.085450,0.264696,0.085150,0.0,0.0,8,9126.125000
2,69039685746313,0.044512,0.028056,0.044368,4.0,0.0,4,235984.500000
3,161514654074162,0.093511,0.083326,0.093581,56.0,0.0,58,165162.413793
4,168103949904656,0.087588,0.034847,0.089625,0.0,0.0,10,87199.100000
